<a href="https://colab.research.google.com/github/AkiSugi/sample_AnimeGAN/blob/main/AnimeGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0.はじめに  
test.pyをColaboratoryで動作させるためにファイルを若干修正した  
スタイル生成のみ2021/5/18に動作確認を行った  
オリジナルのスタイルデータの作成は、未確認  
また、スタイルの適用具合の調整ができないかが、残件  


TachibanaYoshinoさんのオリジナルの説明  
https://github.com/TachibanaYoshino/AnimeGAN/blob/master/doc/Japanese_README.md  
ソース 実世界の写真をアニメ画像に変換  
https://github.com/TachibanaYoshino/AnimeGAN  


**画像の注意**  
学習画像は、風景を対象としたようであるので、人物、動物はあまり効果がないと思われる  
乗り物は、それなりに変換されている印象  

**スタイル学習の注意**  
生成されたスタイルデータは、スタイルデータ全体の明るさや階調に影響されるので、夜のアニメ画像をスタイルデータとして選択しないようにし、スタイルデータ全体の明るさと暗さの整合性を促進するために、スタイルデータ全体の露出補正を行う必要がある。

# 1.事前準備  

In [ ]:
!git clone https://github.com/TachibanaYoshino/AnimeGAN

Cloning into 'AnimeGAN'...
remote: Enumerating objects: 1199, done.
remote: Total 1199 (delta 0), reused 0 (delta 0), pack-reused 1199
Receiving objects: 100% (1199/1199), 266.61 MiB | 24.88 MiB/s, done.
Resolving deltas: 100% (338/338), done.


スタイルデータを取得  
作業用フォルダーとして、スタイルを適用する画像ファイルを格納フォルダーoriginalを作成   
 

In [ ]:
%cd /content/AnimeGAN
!mkdir original

/content/AnimeGAN
--2021-05-18 08:29:53--  https://github.com/TachibanaYoshino/AnimeGAN/releases/download/Haoyao-style_V1.0/Haoyao-style.zip
Resolving github.com (github.com)... 13.114.40.48
Connecting to github.com (github.com)|13.114.40.48|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2021-05-18 08:29:53 ERROR 404: Not Found.

unzip:  cannot find or open Haoyao-style.zip, Haoyao-style.zip.zip or Haoyao-style.zip.ZIP.


# 2.初期化  
最初に実行するとき、あるいは、ランタイムを再起動した場合に、以下のCellから実行すること  

In [ ]:
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

TensorFlow 1.x selected.
1.15.2


モジュールの読み込みと関数の定義  
 with tf.variable_scope("generator", reuse=tf.AUTO_REUSE): オリジナルでは、False  

---

 読み込み時に縦横32の倍数になるようリサイズを行う  
 256,256は最小サイズ  

---

 if_adjust_brightness がTrueの場合、生成された画像の明るさを元の画像を参考に調整を行う  
  OpenCVで輝度画像を作成し、平均値を求め、同じ値になるよう補正している  
  255のClip処理有  

In [ ]:
%cd /content/AnimeGAN

import os
import time
import numpy as np

from tools.utils import *
from glob import glob

from net import generator

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

def stats_graph(graph):
    flops = tf.profiler.profile(graph, options=tf.profiler.ProfileOptionBuilder.float_operation())
    # params = tf.profiler.profile(graph, options=tf.profiler.ProfileOptionBuilder.trainable_variables_parameter())
    print('FLOPs: {}'.format(flops.total_float_ops))

def test(checkpoint_dir, style_name, test_files, if_adjust_brightness, img_size=[256,256]):
    # tf.reset_default_graph()

    result_dir = 'results/'+style_name
    check_folder(result_dir)

    # test_real = tf.placeholder(tf.float32, [1, 256, 256, 3], name='test')
    test_real = tf.placeholder(tf.float32, [1, None, None, 3], name='test')

    with tf.variable_scope("generator", reuse=tf.AUTO_REUSE):
        Anime_generated = generator.G_net(test_real).fake

    saver = tf.train.Saver()

    gpu_options = tf.GPUOptions(allow_growth=True)
    with tf.Session(config=tf.ConfigProto(allow_soft_placement=True, gpu_options=gpu_options)) as sess:
        # tf.global_variables_initializer().run()
        # load model
        ckpt = tf.train.get_checkpoint_state(checkpoint_dir)  # checkpoint file information
        if ckpt and ckpt.model_checkpoint_path:
            ckpt_name = os.path.basename(ckpt.model_checkpoint_path)  # first line
            saver.restore(sess, os.path.join(checkpoint_dir, ckpt_name))
            print("\nSuccess to read {}\n".format(ckpt_name))
        else:
            print("\nFailed to find a checkpoint")
            return None
        
        # FLOPs
        # stats_graph(tf.get_default_graph())

        begin = time.time()
        for sample_file in test_files :
            print('Processing image: ' + sample_file)
            sample_image = np.asarray(load_test_data(sample_file, img_size))
            fname = os.path.basename(sample_file)
            save_path = os.path.join(result_dir,'{0}'.format('anime_'+fname))

            anime_img = sess.run(Anime_generated, feed_dict = {test_real : sample_image})
            anime = inverse_transform(anime_img.squeeze())

            # Trueを指定
            if if_adjust_brightness:
              org_img = read_img(sample_file)
              anime_adj = adjust_brightness_from_src_to_dst(anime, org_img)
              cv2.imwrite(save_path, cv2.cvtColor(anime_adj, cv2.COLOR_BGR2RGB))
              print(anime_adj.shape)
            else:
              cv2.imwrite(save_path, cv2.cvtColor(anime, cv2.COLOR_BGR2RGB))
              anime_img = anime

        end = time.time()
        print(f'\nTotal time: {end-begin} s')
        print(f'Average time : {(end-begin)/len(test_files)} s')

    return anime_img

/content/AnimeGAN


# 3.Animeスタイルを適用した画像の生成  
originalフォルダーの画像は、一括でスタイル適用された画像が生成される    
画像を追加した場合、再度このcellから実行するだけでよい

スタイルを適用させる画像のサイズが大きいとGPUのリソースの枯渇でエラーとなる可能性がある  
2016x1512はOK 12秒    
4032x3024はエラー  

In [ ]:
%cd /content/AnimeGAN

org_files = glob('{}/*.*'.format('original'))
st_img = test('checkpoint/generator_Hayao_weight', 'Hayao', org_files, True)

/content/AnimeGAN
INFO:tensorflow:Restoring parameters from checkpoint/generator_Hayao_weight/Hayao-60.ckpt

Success to read Hayao-60.ckpt

Processing image: original/PXL_20210327_024814022_2048.jpg
(1152, 2048, 3)
Processing image: original/PXL_20201220_082233027.NIGHT_2048.jpg
(1152, 2048, 3)

Total time: 7.405293703079224 s
Average time : 3.702646851539612 s


# 4.ユーティリティ
/cotentの.jpgのファイルを縮小して、originalへ保存  
results/Hayaoのデータをすべてダウンロード  


In [ ]:
import cv2
MAXSIZE=1280

org_files = glob('{}/*.jpg'.format('/content'))
print(org_files)

for file in org_files :
    img = cv2.imread(file)
    print(img.shape)
    h0 = img.shape[0]
    w0 = img.shape[1]

    if w0 > h0 :
        if w0 > MAXSIZE :
            w1 = MAXSIZE
            h1 = int(h0 * MAXSIZE / w0)
            img = cv2.resize(img, (w1, h1))
    else :
        if h1 > MAXSIZE :
            h1 = MAXSIZE
            w1 = int(w0 * MAXSIZE / h0)
            img = cv2.resize(img, (w1, h1))

    print (img.shape)
    fname = os.path.basename(file)
    save_path = os.path.join('/content/AnimeGAN/original','{0}'.format('s_'+fname))
    print(save_path)
    cv2.imwrite(save_path, img)

In [ ]:
from google.colab import files

org_files = glob('{}/*.*'.format('/content/AnimeGAN/results/Hayao'))
print(org_files)
for file in org_files :
    files.download(file)

In [ ]:
!rm /content/AnimeGAN/original/*.*
!rm /content/AnimeGAN/results/Hayao/*.*

# 5.補足とメモ  
以下はオリジナルのtest.pyを使用する場合  
スタイル適用画像は、sampleに保存  
結果は、results/Hayaoに出力される  

In [ ]:
!python test.py --checkpoint_dir checkpoint/generator_Hayao_weight --test_dir original --style_name Hayao

In [ ]:
!pip install tensorflow-gpu==1.8.0

In [ ]:
%cd /content
!wget https://github.com/TachibanaYoshino/AnimeGAN/releases/download/Hayao_styleDataV2/Hayao_styleDataV2.7z
!7z x Hayao_styleDataV2.7z

In [ ]:
!7z x Hayao_styleDataV2.7z

https://www.youtube.com/watch?v=R-DBiElq7OQ  
https://www.youtube.com/channel/UCaZuPdmZ380SFUMKHVsv_AA  

